In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;


Tendencia mensual (24 meses)

Construye month_start con make_date(year, month, 1) desde fact_sales.

Agrega ventas y botellas por mes (SUM(sale_dollars), SUM(sale_bottles)).

Filtra los últimos 24 meses y ordena cronológicamente.

In [0]:
%sql
SELECT
  make_date(CAST(year AS INT), CAST(month AS INT), 1) AS month_start,
  SUM(sale_dollars) AS sales_usd,
  SUM(sale_bottles) AS bottles
FROM fact_sales
WHERE make_date(CAST(year AS INT), CAST(month AS INT), 1)
      >= add_months(date_trunc('month', current_date()), -24)
GROUP BY make_date(CAST(year AS INT), CAST(month AS INT), 1)
ORDER BY month_start;


month_start,sales_usd,bottles
2023-10-01,1184661.83,63943
2023-11-01,829290.77,55803
2024-01-01,1633059.12,117590
2024-04-01,2277.00,132
2024-07-01,742.50,66
2024-08-01,1793234.74,124135
2024-10-01,132.00,44
2024-11-01,1919173.41,128073
2025-04-01,1451333.29,116908
2025-05-01,3567.96,264


Tendencia mensual con YoY (24 meses)

Agrega ventas por mes (sales_usd) desde fact_sales.

Calcula ventas del mismo mes del año anterior con LAG(..., 12).

Deriva YoY: (sales - sales_prev_year) / sales_prev_year.

Filtra últimos 24 meses y ordena por month_start.

In [0]:
%sql
WITH m AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS month_start,
    SUM(sale_dollars) AS sales_usd
  FROM fact_sales
  GROUP BY make_date(CAST(year AS INT), CAST(month AS INT), 1)
),
w AS (
  SELECT
    month_start,
    sales_usd,
    LAG(sales_usd, 12) OVER (ORDER BY month_start) AS sales_prev_year
  FROM m
)
SELECT
  month_start,
  sales_usd,
  sales_prev_year,
  (sales_usd - sales_prev_year) / NULLIF(sales_prev_year, 0) AS yoy_pct
FROM w
WHERE month_start >= add_months(date_trunc('month', current_date()), -24)
ORDER BY month_start;


month_start,sales_usd,sales_prev_year,yoy_pct
2023-10-01,1184661.83,107728341.93,-0.989003248274537
2023-11-01,829290.77,146880836.41,-0.994353989327204
2024-01-01,1633059.12,131333616.81,-0.987565566534556
2024-04-01,2277.00,159570848.91,-0.999985730476365
2024-07-01,742.50,154065910.57,-0.999995180634072
2024-08-01,1793234.74,63180818.69,-0.971617418432031
2024-10-01,132.00,23732618.14,-0.999994438034640
2024-11-01,1919173.41,7357826.52,-0.739165716290903
2025-04-01,1451333.29,5925844.10,-0.755084125483490
2025-05-01,3567.96,3340321.71,-0.998931851387452


Ranking de condados (12 meses)

Suma ventas y botellas por county desde fact_sales.

Filtra los últimos 12 meses usando year/month.

Ordena por sales_usd descendente y devuelve el Top 20.

In [0]:
%sql
SELECT
  county,
  SUM(sale_dollars) AS sales_usd,
  SUM(sale_bottles) AS bottles
FROM fact_sales
WHERE make_date(CAST(year AS INT), CAST(month AS INT), 1)
      >= add_months(date_trunc('month', current_date()), -12)
GROUP BY county
ORDER BY sales_usd DESC
LIMIT 20;


county,sales_usd,bottles
POLK,589184.20,46211
LINN,320883.42,23529
SCOTT,258434.00,17534
DUBUQUE,242359.26,16698
WOODBURY,181145.03,12485
DALLAS,177586.64,7898
BLACK HAWK,149521.13,11583
POTTAWATTAMIE,118879.20,12331
STORY,109003.51,3685
CERRO GORDO,90148.41,6677


Ranking de tiendas (últimos 3 meses relativos al último mes cargado)

Construye ym (año-mes) desde fact_sales y toma el máximo mes disponible (max_ym).

Une con dim_store para traer nombre, ciudad y condado de la tienda.

Suma ventas y botellas por tienda en la ventana max_ym − 3 meses … max_ym.

Ordena por sales_usd descendente y devuelve el Top 20.

In [0]:
%sql
WITH f AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS ym,
    store_id,
    sale_dollars,
    sale_bottles
  FROM fact_sales
),
maxm AS (SELECT max(ym) AS max_ym FROM f)
SELECT
  f.store_id,
  s.store_name,
  s.city,
  s.county,
  SUM(f.sale_dollars) AS sales_usd,
  SUM(f.sale_bottles) AS bottles
FROM f
LEFT JOIN dim_store s ON f.store_id = s.store_id
CROSS JOIN maxm
WHERE f.ym >= add_months(max_ym, -3)   -- cambiá a -12 si querés 1 año
GROUP BY f.store_id, s.store_name, s.city, s.county
ORDER BY sales_usd DESC
LIMIT 20;




store_id,store_name,city,county,sales_usd,bottles
4829,CENTRAL CITY 2,DES MOINES,1022200,47634.18,2123
3773,BENZ DISTRIBUTING,CEDAR RAPIDS,1082100,43795.73,1364
2633,HY-VEE #3 / BDI / DES MOINES,DES MOINES,1062200,33817.63,2189
6344,HELLO MART LLC / SIOUX CITY,SIOUX CITY,1012100,32012.64,1210
2614,HY-VEE #3 FOOD & DRUGSTORE / DAVENPORT,DAVENPORT,1062200,30671.52,1584
2613,HY-VEE FOOD STORE #1 / COUNCIL BLUFFS,COUNCIL BLUFFS,1081600,25558.17,2178
2644,HY-VEE FORT DODGE WINE AND SPIRITS,FORT DODGE,1011200,19269.25,1133
5351,DOWNTOWN LIQUOR,SIOUX CITY,1012100,18681.96,1848
5916,ANOTHER ROUND / DEWITT,DEWITT,1022200,18511.68,858
10193,BEER ON FLOYD / SIOUX CITY,SIOUX CITY,1031100,18354.05,693


Categorías más vendidas (últimos 12 meses relativos al último mes cargado)

Construye ym (año-mes) y detecta max_ym en fact_sales.

Une con dim_item para obtener category_name.

Agrega ventas y botellas por categoría en la ventana max_ym − 12 meses … max_ym.

Ordena por sales_usd descendente y devuelve Top 20.

In [0]:
%sql
WITH f AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS ym,
    item_no,
    sale_dollars,
    sale_bottles
  FROM fact_sales
),
maxm AS (SELECT max(ym) AS max_ym FROM f)
SELECT
  i.category_name,
  SUM(f.sale_dollars) AS sales_usd,
  SUM(f.sale_bottles) AS bottles
FROM f
LEFT JOIN dim_item i ON f.item_no = i.item_no
CROSS JOIN maxm
WHERE f.ym >= add_months(max_ym, -12)
GROUP BY i.category_name
ORDER BY sales_usd DESC
LIMIT 20;


category_name,sales_usd,bottles
AMERICAN VODKAS,890773.07,73975
CANADIAN WHISKIES,506953.37,30569
STRAIGHT BOURBON WHISKIES,467786.99,20295
100% AGAVE TEQUILA,366903.57,12320
WHISKEY LIQUEUR,296147.61,65527
TENNESSEE WHISKIES,249613.43,8921
SPICED RUM,224905.34,16170
IMPORTED CORDIALS & LIQUEURS,210397.11,8899
MIXTO TEQUILA,205735.20,15059
TEMPORARY & SPECIALTY PACKAGES,171974.00,5687


Crecimiento YoY por categoría (últimos 12m vs. 12m previos)

Construye ym y detecta max_ym (último mes disponible).

Separa ventas en dos ventanas: curr (últimos 12m) vs prev (12m anteriores).

Agrega ventas por category_name en cada ventana y calcula YoY: (sales_curr - sales_prev) / sales_prev.

Ordena por mayor crecimiento y limita al Top 20.

In [0]:
%sql
WITH f AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS ym,
    item_no,
    sale_dollars
  FROM fact_sales
),
maxm AS (SELECT max(ym) AS max_ym FROM f),
w AS (
  SELECT
    item_no,
    CASE
      WHEN f.ym BETWEEN add_months(m.max_ym, -11) AND m.max_ym THEN 'curr'
      WHEN f.ym BETWEEN add_months(m.max_ym, -23) AND add_months(m.max_ym, -12) THEN 'prev'
    END AS win,
    sale_dollars
  FROM f CROSS JOIN maxm m
  WHERE f.ym >= add_months(m.max_ym, -23)
),
by_cat AS (
  SELECT i.category_name, win, SUM(sale_dollars) AS sales
  FROM w
  LEFT JOIN dim_item i ON w.item_no = i.item_no
  GROUP BY i.category_name, win
),
final AS (
  SELECT
    category_name,
    SUM(CASE WHEN win='curr' THEN sales ELSE 0 END) AS sales_curr,
    SUM(CASE WHEN win='prev' THEN sales ELSE 0 END) AS sales_prev
  FROM by_cat
  GROUP BY category_name
)
SELECT
  category_name,
  sales_curr,
  sales_prev,
  (sales_curr - sales_prev) / NULLIF(sales_prev, 0) AS yoy_growth
FROM final
ORDER BY yoy_growth DESC NULLS LAST
LIMIT 20;


category_name,sales_curr,sales_prev,yoy_growth
NEUTRAL GRAIN SPIRITS,2945.25,445.50,5.611111
GOLD RUM,8711.89,1740.86,4.004360
SINGLE BARREL BOURBON WHISKIES,17726.17,4042.50,3.384952
MIXTO TEQUILA,205735.20,50471.08,3.076299
CORN WHISKIES,2260.50,709.50,2.186047
TENNESSEE WHISKIES,249613.43,80259.96,2.110062
COFFEE LIQUEURS,33004.84,12122.99,1.722500
IMPORTED VODKAS,151060.25,61082.56,1.473050
COCKTAILS/RTD,84572.62,38383.62,1.203352
BLENDED WHISKIES,144157.42,76539.54,0.883437


Margen bruto por ítem (últimos 6 meses)

Construye ym y toma max_ym; filtra ventana max_ym−6 … max_ym.

Une con dim_item para descripción del ítem.

Calcula revenue (SUM(sale_dollars)), cost (SUM(sale_bottles*state_bottle_cost)), gross_margin (revenue−cost) y gm_pct.

Considera solo filas con sale_bottles > 0, ordena por gross_margin_usd y limita a 50.

In [0]:
%sql
WITH f AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS ym,
    item_no,
    sale_bottles,
    sale_dollars,
    state_bottle_cost
  FROM fact_sales
),
maxm AS (SELECT max(ym) AS max_ym FROM f)
SELECT
  i.item_no,
  i.item_desc,
  SUM(f.sale_dollars)                                       AS revenue_usd,
  SUM(f.sale_bottles * f.state_bottle_cost)                 AS cost_usd,
  SUM(f.sale_dollars - f.sale_bottles * f.state_bottle_cost) AS gross_margin_usd,
  SUM(f.sale_dollars - f.sale_bottles * f.state_bottle_cost)
    / NULLIF(SUM(f.sale_dollars), 0)                        AS gm_pct
FROM f
LEFT JOIN dim_item i ON f.item_no = i.item_no
CROSS JOIN maxm
WHERE f.ym >= add_months(max_ym, -6)
  AND f.sale_bottles > 0
GROUP BY i.item_no, i.item_desc
ORDER BY gross_margin_usd DESC
LIMIT 50;





item_no,item_desc,revenue_usd,cost_usd,gross_margin_usd,gm_pct
38178,TITOS HANDMADE VODKA,190294.50,126863.00,63431.50,0.333333
43337,CAPTAIN MORGAN ORIGINAL SPICED,90686.97,60442.47,30244.50,0.333504
921542,STEEPLE RIDGE BOURBON SINGLE BARREL CASK STRENGTH BARREL BUY,71280.00,47520.00,23760.00,0.333333
10785,CROWN ROYAL BLACKBERRY,65809.92,43864.92,21945.00,0.333460
10807,CROWN ROYAL REGAL APPLE,55130.24,36746.49,18383.75,0.333460
38176,TITOS HANDMADE VODKA,48015.00,32010.00,16005.00,0.333333
911070,MEMBERS MARK CANADIAN WHISKY,43560.00,29040.00,14520.00,0.333333
38177,TITOS HANDMADE VODKA,41733.12,27815.04,13918.08,0.333502
64870,FIREBALL CINNAMON WHISKEY,41200.50,27467.00,13733.50,0.333333
88296,PATRON SILVER,34593.68,21355.18,13238.50,0.382686


Margen bruto por ítem (últimos 6 meses)

Construye ym y toma max_ym; filtra ventana max_ym−6 … max_ym.

Une con dim_item para descripción del ítem.

Calcula revenue (SUM(sale_dollars)), cost (SUM(sale_bottles*state_bottle_cost)), gross_margin (revenue−cost) y gm_pct.

Considera solo filas con sale_bottles > 0, ordena por gross_margin_usd y limita a 50.

In [0]:
%sql
WITH f AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS ym,
    item_no,
    sale_bottles,
    sale_dollars,
    state_bottle_cost
  FROM fact_sales
),
maxm AS (SELECT max(ym) AS max_ym FROM f)
SELECT
  i.category_name,
  SUM(f.sale_dollars)                                    AS revenue_usd,
  SUM(f.sale_bottles * f.state_bottle_cost)              AS cost_usd,
  SUM(f.sale_dollars - f.sale_bottles * f.state_bottle_cost) AS gross_margin_usd,
  SUM(f.sale_dollars - f.sale_bottles * f.state_bottle_cost)
    / NULLIF(SUM(f.sale_dollars), 0)                     AS gm_pct
FROM f
LEFT JOIN dim_item i ON f.item_no = i.item_no
CROSS JOIN maxm
WHERE f.ym >= add_months(max_ym, -12)
  AND f.sale_bottles > 0
GROUP BY i.category_name
ORDER BY gross_margin_usd DESC
LIMIT 50;


category_name,revenue_usd,cost_usd,gross_margin_usd,gm_pct
AMERICAN VODKAS,890773.07,594034.76,296738.31,0.333124
CANADIAN WHISKIES,506953.37,337880.18,169073.19,0.333508
STRAIGHT BOURBON WHISKIES,467786.99,311842.08,155944.91,0.333367
100% AGAVE TEQUILA,366903.57,241605.76,125297.81,0.341501
WHISKEY LIQUEUR,296147.61,197422.61,98725.00,0.333364
TENNESSEE WHISKIES,249613.43,166397.11,83216.32,0.333381
SPICED RUM,224905.34,150438.75,74466.59,0.331102
IMPORTED CORDIALS & LIQUEURS,210397.11,140036.16,70360.95,0.334420
MIXTO TEQUILA,205735.20,137108.18,68627.02,0.333570
TEMPORARY & SPECIALTY PACKAGES,171974.00,114636.28,57337.72,0.333409


Margen bruto por vendor (últimos 12 meses)

Construye ym y obtiene max_ym; filtra ventana max_ym−12 … max_ym.

Une con dim_item para mapear vendor_no/vendor_name.

Calcula revenue (SUM(sale_dollars)), cost (SUM(sale_bottles*state_bottle_cost)), gross_margin y gm_pct.

Considera solo filas con sale_bottles > 0, agrupa por vendor y ordena por gross_margin_usd (Top 50).

In [0]:
%sql
WITH f AS (
  SELECT
    make_date(CAST(year AS INT), CAST(month AS INT), 1) AS ym,
    item_no,
    sale_bottles,
    sale_dollars,
    state_bottle_cost
  FROM fact_sales
),
maxm AS (SELECT max(ym) AS max_ym FROM f)
SELECT
  i.vendor_no,
  i.vendor_name,
  SUM(f.sale_dollars)                                    AS revenue_usd,
  SUM(f.sale_bottles * f.state_bottle_cost)              AS cost_usd,
  SUM(f.sale_dollars - f.sale_bottles * f.state_bottle_cost) AS gross_margin_usd,
  SUM(f.sale_dollars - f.sale_bottles * f.state_bottle_cost)
    / NULLIF(SUM(f.sale_dollars), 0)                     AS gm_pct
FROM f
LEFT JOIN dim_item i ON f.item_no = i.item_no
CROSS JOIN maxm
WHERE f.ym >= add_months(max_ym, -12)
  AND f.sale_bottles > 0
GROUP BY i.vendor_no, i.vendor_name
ORDER BY gross_margin_usd DESC
LIMIT 50;


vendor_no,vendor_name,revenue_usd,cost_usd,gross_margin_usd,gm_pct
421,SAZERAC COMPANY INC,967074.02,644692.07,322381.95,0.333358
260,DIAGEO AMERICAS,942727.06,627828.08,314898.98,0.334030
301,FIFTH GENERATION INC,485191.30,323447.74,161743.56,0.333360
65,JIM BEAM BRANDS,339776.69,226494.18,113282.51,0.333403
35,BACARDI USA INC,281864.88,185875.91,95988.97,0.340550
85,BROWN FORMAN CORP.,263275.98,175503.46,87772.52,0.333386
259,HEAVEN HILL BRANDS,253442.53,169502.19,83940.34,0.331201
370,PERNOD RICARD USA,227788.22,151824.64,75963.58,0.333483
395,PROXIMO,203788.97,135810.84,67978.13,0.333571
434,LUXCO INC,163599.59,109055.10,54544.49,0.333402


Tendencia anual con YoY

Agrega ventas por año desde fact_sales (SUM(sale_dollars) agrupado por year).

Calcula ventas del año previo con LAG(..., 1).

Deriva YoY: (sales_usd - prev_year_sales) / prev_year_sales.

Ordena por year para ver la evolución cronológica.

In [0]:
%sql
WITH y AS (
  SELECT CAST(year AS INT) AS yy, SUM(sale_dollars) AS sales_usd
  FROM fact_sales
  GROUP BY CAST(year AS INT)
)
SELECT
  yy AS year,
  sales_usd,
  LAG(sales_usd) OVER (ORDER BY yy) AS prev_year_sales,
  (sales_usd - LAG(sales_usd) OVER (ORDER BY yy))
    / NULLIF(LAG(sales_usd) OVER (ORDER BY yy), 0)       AS yoy_pct
FROM y
ORDER BY yy;


year,sales_usd,prev_year_sales,yoy_pct
2012,1149401845.35,null,null
2013,1162273868.47,1149401845.35,0.011198888510641
2014,1221653116.42,1162273868.47,0.051088860862170
2015,1278871863.28,1221653116.42,0.046837147215469
2016,1491076803.48,1278871863.28,0.165931354260735
2017,1839021494.75,1491076803.48,0.233351287108711
2018,1911370238.69,1839021494.75,0.039340890874054
2019,1583114432.13,1911370238.69,-0.171738473224830
2020,1581390811.77,1583114432.13,-0.001088752856407
2021,1726423543.68,1581390811.77,0.091712137714819


Tendencia diaria (últimos 60 días, desde Silver strict)

Usa silver.iowa_clean_v2_strict y castea sale_date a día (d).

Define ventana máximo día −59 … máximo día vía maxd.

Agrega ventas y botellas por día; orden cronológico para ver la tendencia.

In [0]:
%sql
WITH s AS (
  SELECT
    CAST(sale_date AS DATE) AS d,
    sale_dollars,
    sale_bottles
  FROM silver.iowa_clean_v2_strict
  WHERE sale_date IS NOT NULL
),
maxd AS (SELECT MAX(d) AS max_d FROM s)
SELECT
  d AS day,
  SUM(sale_dollars) AS sales_usd,
  SUM(sale_bottles) AS bottles
FROM s
CROSS JOIN maxd
WHERE d BETWEEN date_add(max_d, -59) AND max_d
GROUP BY d
ORDER BY day;


day,sales_usd,bottles
2025-04-18,375621.51,30096
2025-04-20,33764.61,4125
2025-04-21,657352.63,51678
2025-04-22,384594.54,31009
2025-05-01,198.00,66
2025-05-14,1092.96,66
2025-05-23,2277.00,132


Optimización de precios: correlación precio–volumen por categoría (36 meses)


Desde silver.iowa_clean_v2_strict, consolida mensual por categoría: units, revenue y precio promedio (avg_price_per_bottle).


Toma la ventana de 36 meses relativa al último mes disponible.


Calcula la correlación corr(precio, unidades) por categoría (y muestra meses, precio medio y unidades totales).


Filtra categorías con ≥ 6 meses y ordena por correlación ascendente (más negativa arriba ⇒ mayor sensibilidad del volumen al precio).



In [0]:
%sql
WITH s AS (
  SELECT
    date_trunc('month', CAST(sale_date AS DATE)) AS ym,
    category_name,
    sale_bottles,
    sale_dollars
  FROM silver.iowa_clean_v2_strict
  WHERE sale_date IS NOT NULL
),
m AS (
  SELECT
    ym,
    category_name,
    SUM(sale_bottles) AS units,
    SUM(sale_dollars) AS revenue,
    SUM(sale_dollars) / NULLIF(SUM(sale_bottles), 0) AS avg_price_per_bottle
  FROM s
  GROUP BY ym, category_name
),
maxm AS (SELECT MAX(ym) AS max_ym FROM m),
win AS (SELECT * FROM m, maxm WHERE ym >= add_months(max_ym, -36))
SELECT
  category_name,
  COUNT(*)                           AS months,
  corr(avg_price_per_bottle, units)  AS price_volume_corr,
  AVG(avg_price_per_bottle)          AS avg_price_usd,
  SUM(units)                         AS total_units
FROM win
GROUP BY category_name
HAVING COUNT(*) >= 6
ORDER BY price_volume_corr ASC NULLS LAST
LIMIT 20;


category_name,months,price_volume_corr,avg_price_usd,total_units
COFFEE LIQUEURS,15,-0.4829948838846731,21.6315722781916148180667,91729
AMERICAN SLOE GINS,8,-0.4158428580638389,8.2325000000000000000000,3080
AMERICAN DISTILLED SPIRITS SPECIALTY,15,-0.3722878164263218,23.5296056781940731056000,65186
MIXTO TEQUILA,15,-0.3299676818172164,14.3686640763223296254667,816607
FLAVORED GIN,15,-0.3046855389399345,24.7123882673736172901333,34331
AMERICAN CORDIALS & LIQUEURS,15,-0.2827045993268137,10.9762666256296506622000,520014
AMERICAN BRANDIES,15,-0.2654135986082187,6.9480261531012072557333,720280
STRAIGHT BOURBON WHISKIES,15,-0.2641461952109174,21.8893746012699623552000,1602678
TEMPORARY & SPECIALTY PACKAGES,15,-0.24125994342532442,30.4778068177012937944667,382228
IMPORTED CORDIALS & LIQUEURS,15,-0.2255052846997981,24.2270723749864452447333,583143


Optimización de precios: elasticidad precio–demanda por categoría (36 meses)

Desde silver.iowa_clean_v2_strict, consolida mensual por categoría: units, revenue y precio promedio (avg_price_per_bottle).

Calcula la pendiente 
𝑑
𝑄
/
𝑑
𝑃
dQ/dP como covar_samp(price, units) / var_samp(price) y la elasticidad ≈ pendiente * (P̄/Q̄).

Incluye también la correlación corr(price, units) como señal rápida de sensibilidad.

Usa ventana últimos 36 meses (relativa al máximo mes) y filtra categorías con ≥ 6 meses.

Ordena por elasticidad ascendente (más negativa ⇒ demanda más elástica).

In [0]:
%sql
WITH s AS (
  SELECT
    date_trunc('month', CAST(sale_date AS DATE)) AS ym,
    category_name,
    sale_bottles,
    sale_dollars
  FROM silver.iowa_clean_v2_strict
  WHERE sale_date IS NOT NULL
),
m AS (  -- mensual por categoría
  SELECT
    ym, category_name,
    SUM(sale_bottles) AS units,
    SUM(sale_dollars) AS revenue,
    SUM(sale_dollars)/NULLIF(SUM(sale_bottles),0) AS avg_price_per_bottle
  FROM s
  GROUP BY ym, category_name
),
maxm AS (SELECT MAX(ym) AS max_ym FROM m),
win AS (SELECT * FROM m, maxm WHERE ym >= add_months(max_ym, -36))
SELECT
  category_name,
  COUNT(*) AS months,
  -- pendiente d(units)/d(price)
  covar_samp(avg_price_per_bottle, units) / NULLIF(var_samp(avg_price_per_bottle),0) AS slope_units_per_usd,
  -- elasticidad ≈ slope * (P/Q)
  (covar_samp(avg_price_per_bottle, units) / NULLIF(var_samp(avg_price_per_bottle),0))
    * (AVG(avg_price_per_bottle) / NULLIF(AVG(units),0))                                   AS price_elasticity,
  corr(avg_price_per_bottle, units)                                                        AS corr_p_u,
  AVG(avg_price_per_bottle)                                                                AS avg_price_usd,
  SUM(units)                                                                               AS total_units,
  SUM(revenue)                                                                             AS revenue_usd
FROM win
GROUP BY category_name
HAVING COUNT(*) >= 6
ORDER BY price_elasticity ASC NULLS LAST
LIMIT 20;


category_name,months,slope_units_per_usd,price_elasticity,corr_p_u,avg_price_usd,total_units,revenue_usd
AMERICAN SLOE GINS,8,-1001.6260162601661,-21.417886178861863,-0.4158428580638389,8.2325000000000000000000,3080,25103.54
NEUTRAL GRAIN SPIRITS FLAVORED,15,-7480.492258257545,-10.740240866575194,-0.1632154597503011,16.7557789848391340025333,175054,2910081.13
COFFEE LIQUEURS,15,-1868.471929909636,-6.609357826033065,-0.4829948838846731,21.6315722781916148180667,91729,1774716.13
STRAIGHT BOURBON WHISKIES,15,-26568.60159500886,-5.443109030120656,-0.2641461952109175,21.8893746012699623552000,1602678,33603440.97
MIXTO TEQUILA,15,-18973.425324817963,-5.007722959811075,-0.3299676818172164,14.3686640763223296254667,816607,10879835.01
IMPORTED SCHNAPPS,15,-6383.87086470966,-4.472987596554379,-0.17052364160201433,17.4073551620165538696000,372658,6358223.30
FLAVORED GIN,15,-368.8477092795891,-3.9825993139940086,-0.3046855389399345,24.7123882673736172901333,34331,801974.47
IMPORTED CORDIALS & LIQUEURS,15,-5392.649118027876,-3.360619105657656,-0.22550528469979808,24.2270723749864452447333,583143,13413302.32
AMERICAN BRANDIES,15,-18844.92972313228,-2.7267534410856347,-0.2654135986082187,6.9480261531012072557333,720280,4557782.02
CREAM LIQUEURS,15,-2541.02440724838,-2.708541554365316,-0.14370602655116857,20.1844987659428321296667,284042,5583957.50
